In [ ]:
db_path = "benchmark.db"
measure = "cpu_time"

In [ ]:
from reprobench.utils import init_db
init_db(db_path)

In [ ]:
import itertools

import pandas as pd
import seaborn as sns

from reprobench.core.db import Run, Tool, ParameterGroup
from reprobench.executors.db import RunStatistic


def cactus_plot(measure, **kwargs):
    cactus_df = pd.DataFrame()
    
    for group in ParameterGroup.select():
        tool_name = f"{group.tool_id}_{group.name}"
        measure_field = getattr(RunStatistic, measure)
        values_query = (
            RunStatistic
            .select(measure_field)
            .join(Run)
            .where(Run.tool_id == group.tool_id)
            .where(Run.parameter_group_id == group.id)
            .order_by(measure_field)
        )
        series = pd.Series(
            data=[*itertools.chain.from_iterable(values_query.tuples())],
            name=tool_name,
        ).sort_values()
        cactus_df = cactus_df.append(series, sort=False)
    
    cactus_df = cactus_df.transpose().reset_index(drop=True)
    
    return sns.scatterplot(data=cactus_df, **kwargs)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

fig, ax = plt.subplots(figsize=(8, 6))
plt.xticks()
plt.xlabel("Instance solved")
plt.ylabel("Time (s)")
cactus_plot(measure, ax=ax)
plt.show()